origin: http://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html  
translator: Hongpu Liu

In [1]:
%matplotlib inline
from __future__ import print_function, division

# 序列模型与LSTM网络
现在我们已经看到了多种前馈网络，这些网络不会保持其状态，这也许不是我们想要的。序列模型是自然语言处理的中心：在你的输入中存在某些时序上的依赖关系。最经典的序列模型是用于词性标注的**隐马尔科夫模型**，另一个例子是**条件随机场**。

循环神经网络是一种维护某种状态的网络。例如，它的输出可以作为下一次输入的成员，这样当序列数据进入网络时，信息随之传播。在**LSTM**中，序列中的每一个元素，都有一个对应的**隐藏状态**$h_t$，这个隐藏状态包含了序列之前任何一个点的信息。我们可以将隐藏状态用于语言模型单词预测、词性标注以及大量其他的目的。

## 1. PyTorch中的LSTM
在开始我们的例子之前，注意一下几点。PyTorch中的LSTM的输入需要是一个3D张量。这些张量各个维度所表示的语义是非常重要的。第一个轴是序列自身，第二个轴是小批量中的样本，第三个轴是输入元素。我们还没有讨论小批量，因此暂时忽略它而假设第二个轴的维度是总是1.如果我们要把句子“The cow jumped”送入序列模型，我们的输入将如下所示：

$$\begin{bmatrix}
\overbrace{q_\text{The}}^\text{row vector} \\
q_\text{cow} \\
q_\text{jumped}
\end{bmatrix}$$

要记住该输入的第二个轴的维度是1。

此外，也可以把序列元素依次送入网络，此时要求第一个轴的维度是1。

我们看下面的例子：

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3) # 输入的维度是3， 输出的维度也是3
inputs = [autograd.Variable(torch.randn((1, 3)))
          for _ in range(5)]

# 初始化隐层状态
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn(1, 1, 3)))
for i in inputs:
    # 每次送入序列中的一个元素。每一步之后，hidden中保存隐藏状态。
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# 此外，我们也可以一次把整个序列送入LSTM。
# LSTM返回的第一个值包含了整个序列的隐藏状态 h_t。第二个返回值则是最近的隐藏状态。
# 注意下面的输出中，hidden 的第一个隐层状态 h_t，与 out 的最后一个隐藏状态是相同的。

# 原因如下：
# out 给了我们整个序列的隐藏状态， hidden 允许我们继续计算序列和反向传播。
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, 3)),
          autograd.Variable(torch.randn(1, 1, 3))) # 清除隐藏状态
out, hidden = lstm(inputs, hidden)
print(out)
print('-' * 20)
print(hidden)

Variable containing:
(0 ,.,.) = 
 -0.0187  0.1713 -0.2944

(1 ,.,.) = 
 -0.3521  0.1026 -0.2971

(2 ,.,.) = 
 -0.3191  0.0781 -0.1957

(3 ,.,.) = 
 -0.1634  0.0941 -0.1637

(4 ,.,.) = 
 -0.3368  0.0959 -0.0538
[torch.FloatTensor of size 5x1x3]

--------------------
(Variable containing:
(0 ,.,.) = 
 -0.3368  0.0959 -0.0538
[torch.FloatTensor of size 1x1x3]
, Variable containing:
(0 ,.,.) = 
 -0.9825  0.4715 -0.0633
[torch.FloatTensor of size 1x1x3]
)


## 2. 例子：用LSTM做词性标注
在这个小节，我们将用LSTM进行词性标注。我们将不使用维特比算法、前后向算法或任何类似的东西，而将其作为留给读者的练习。

模型如下：令输入序列为$w_1,\dots,w_M$，其中$w_i \in V$，$V$是词汇表。同样令$T$为标签集，$y_i$是单词$w_i$的标签。记我们对$w_i$的预测为$\hat{y}_i$。

这是一个结构预测模型，我们输出是一个序列$\hat{y}_1,\dots,\hat{y}_M$，其中$\hat{y}_i \in T$。

在预测时，将句子送入LSTM。将时间步$i$的隐藏状态记为$h_i$。同样，为每个标签指定一个唯一的索引（类似于词嵌入那一节中的word_to_ix）。对于$\hat{y}_i$的预测规则为：

$$\hat{y}_i = \text{arg max}_j \ (\log \text{Softmax}(Ah_i + b))_j$$

即获得隐层状态仿射映射的对数softmax，然后该向量中的最大值就是预测的标签。注意，这就意味着$A$的目标空间维度为$|T|$。

首先准备数据：

In [4]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

training_data = [
    ("The dog ate the apple.".split(),["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# 通常嵌入和隐藏层的维度会选用32或64
# 此出设置的比较小，目的是能够看到训练时权重如何变化
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'Everybody': 5, 'ate': 2, 'that': 7, 'apple.': 4, 'dog': 1, 'read': 6, 'book': 8, 'the': 3, 'The': 0}


创建模型：

In [5]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.word_embeddings =nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM把词嵌入的输出作为输入，输出的隐藏状态的维度为 hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        # 线性层将隐藏状态空间映射到标签空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # 在没有进行任何操作之前，不存在任何的隐藏状态。
        # 可以阅读Pytorch文档，隐层状态维度的由来。
        # 轴的语义为：（层数，小批量，隐层维度）
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

训练模型

In [6]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 首先看一下训练前的分数
# 注意：第ij个元素是单词i在标签j上的分数
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_score = model(inputs)
print(tag_score)

for epoch in range(300):
    for sentence, tags in training_data:
        # 步骤1：由于PyTorch会累加梯度，因此每个样本都需要将梯度归零
        model.zero_grad()
        
        # 同样，还需要清零模型的隐藏状态
        model.hidden = model.init_hidden()
        
        # 步骤2：将句子和标签变为包含词索引的向量
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # 步骤3：运行前馈过程
        tag_scores = model(sentence_in)
        
        # 步骤4：计算损失、梯度、调用optimizer更新权重
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# 看一下训练后的词性
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_score = model(inputs)
# 句子“the dog ate the apple”，i，j 对应着“单词i”在“标签j”上的得分。
# 预测的标签是标签的最大值，可以看到预测的结果是： 0 1 2 0 1
# 因为第一行最大的值下标为1
# 预测结果是：DET NOUN VERB DET NOUN，预测正确
print(tag_score)

Variable containing:
-1.1989 -0.9630 -1.1497
-1.2522 -0.9158 -1.1586
-1.2563 -1.0022 -1.0550
-1.1518 -1.1443 -1.0065
-1.1728 -1.0677 -1.0593
[torch.FloatTensor of size 5x3]

Variable containing:
-0.1902 -1.8654 -3.9957
-4.1051 -0.0263 -4.6590
-4.0204 -3.1797 -0.0614
-0.0372 -4.3504 -3.7448
-4.0387 -0.0348 -4.1001
[torch.FloatTensor of size 5x3]



## 3. 练习：利用字符级别的特征来增强LSTM词性标注
在上面的例子中，每一个词都有一个嵌入，该嵌入作为序列模型的输入。让我们利用单词字符的表示来增强词嵌入。我们认为这将带来巨大的帮助，因为例如后缀这样的字符级别的信息对于词性标注非常的重要。例如，以后缀 *-ly* 结束的单词基本上是英语中的副词。

为了实现这一目标，令$c_w$为单词$w$的字符集别表示。令$x_w$为之前的词嵌入。然后将$x_w$和$c_w$拼接到一起作为序列模型的输入。例如$x_w$的维度为5，$c_w$的维度为3，则LSTM接受的输入维度为8.

要获得字符级别的表示，用一个LSTM输入单词的字符序列，然后用最后的隐层状态作为$c_w$。提示：
- 在你的模型里面将有两个LSTM。原来的LSTM输出POS标签分数，新的LSTM输出每个词字符级别的表示。
- 要把字符输入序列模型，你需要嵌入字符。字符嵌入将作为字符级别LSTM的输入。